# Latihan - Pipeline Analisis Artikel Berita
Tujuannya adalah membuat alat otomatis yang dapat memberikan ringkasan tingkat tinggi (topik) dan juga informasi detail (entitas) dari teks berita apa pun. Ini adalah tugas yang sangat umum di dunia industri, digunakan untuk menganalisis media, laporan keuangan, dan banyak lagi.

Kita akan menggunakan dataset **AG News**, yang berisi ribuan artikel berita yang diklasifikasikan ke dalam 4 kategori: **World, Sports, Business, dan Sci/Tech.**

## Bagian 1: Klasifikasi Topik Artikel

### 1. Memuat Data & Library

In [ ]:
import pandas as pd
import numpy as np
import spacy
from collections import defaultdict

# Library untuk Klasifikasi
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Muat dataset (versi training)
# Dataset ini memiliki kolom: Class Index, Title, Description
# Kita akan menggabungkan Title dan Description
df = pd.read_csv('https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv',
                 names=['label', 'title', 'description'])

# Mapping label dari angka ke teks
label_map = {1: 'World', 2: 'Sports', 3: 'Business', 4: 'Sci/Tech'}
df['topic'] = df['label'].map(label_map)

# Gabungkan title dan description menjadi satu kolom teks
df['text'] = ...

# Pilih kolom yang relevan saja dan ambil sampel agar proses lebih cepat
df = df[['text', 'topic']].sample(10000, random_state=42) # Ambil 10,000 sampel acak

print("Dataset berhasil dimuat dan diproses.")
df.head()

### 2. Vektorisasi dan Pembuatan Model

In [ ]:
# Tentukan Fitur (X) dan Target (y)
X = df['text']
y = df['topic']

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Inisialisasi dan fit TF-IDF Vectorizer
tfidf = ...
X_train_vec = ...
X_test_vec = ...

# Latih model Logistic Regression
classifier = LogisticRegression(max_iter=1000)
print("Melatih model klasifikasi...")
classifier.fit(X_train_vec, y_train)
print("Model selesai dilatih!")

# Evaluasi model
y_pred = classifier.predict(X_test_vec)
print("\nLaporan Klasifikasi:\n")
print(classification_report(y_test, y_pred))

## Bagian 2: Ekstraksi Entitas dengan spaCy

In [ ]:
# Muat model spaCy
nlp = ...

def extract_entities(text):
    """
    Fungsi ini mengambil teks dan mengembalikan kamus entitas yang ditemukan.
    """
    doc = ...
    entities = defaultdict(list)
    for ent in doc.ents:
        entities[ent.label_].append(ent.text)

    # Hapus duplikat
    for key in entities:
        entities[key] = list(set(entities[key]))

    return dict(entities)

# Contoh penggunaan
sample_text = "Elon Musk, CEO of SpaceX, announced a new mission to Mars from their headquarters in California."
print("Contoh Ekstraksi Entitas:")
print(extract_entities(sample_text))

## Bagian 3: Menggabungkan Semuanya

In [ ]:
def analyze_article(article_text):
    """
    Pipeline lengkap:
    1. Mengklasifikasikan topik artikel.
    2. Mengekstrak entitas dari artikel.
    3. Mencetak hasilnya dengan rapi.
    """
    print("--- Menganalisis Artikel ---")

    # 1. Prediksi Topik
    text_vec = ...
    predicted_topic = ...

    # 2. Ekstrak Entitas
    entities_found = ...

    # 3. Tampilkan Hasil
    print(f"\nPrediksi Topik: **{predicted_topic}**\n")
    print("--- Entitas yang Ditemukan ---")
    if not entities_found:
        print("Tidak ada entitas yang ditemukan.")
    else:
        for label, items in entities_found.items():
            print(f"- **{label}**: {', '.join(items)}")

    print("\n--- Analisis Selesai ---\n")

# Uji Coba Pipeline Lengkap

In [ ]:
# Contoh 1: Artikel Bisnis/Teknologi
article_1 = """
Microsoft Corp on Tuesday announced its next-generation Surface laptops,
including a new model with a custom artificial intelligence chip, as it amps up its rivalry
with Apple Inc ahead of the holiday shopping season in the United States.
Satya Nadella presented the new features in a conference in New York.
"""
analyze_article(article_1)

# Contoh 2: Artikel Olahraga
article_2 = """
Real Madrid secured a dramatic late victory against Manchester City in the Champions League final
held in Istanbul. A stunning goal from Vinicius Junior in the 88th minute sealed the win for
the Spanish giants, leaving manager Pep Guardiola disappointed.
"""
analyze_article(article_2)